In [1]:
import pyodbc

In [3]:
## Hotel
class Hotel:
    menu={}
    conn=pyodbc.connect('Driver={SQL Server};'
        'Server=DAVIDG\SQLEXPRESS01;'
        'Database=Hotel_db;'
        'Trusted_Connection=yes')
    def __init__(self):
        print("Welcome to the Hotel Spice")
        self.cursor=self.conn.cursor()
        self.cursor.execute('Select * from dbo.MENU')
    def Show_menu(self):
        records= self.cursor.fetchall()
        for row in records:
            self.menu[row.Menu_id] = {
            "name": row.ItemName,
            "half": row.Half_price,
            "full": row.Full_price}
        print("--Menu--")
        for Mid,item in self.menu.items():
            print(f"{Mid}. {item['name']} | Half: ₹{item['half']} | Full: ₹{item['full']}")
    def captain(self):
        print("What would you like to have: ")

<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:5: SyntaxWarning: invalid escape sequence '\S'
C:\Users\Asus\AppData\Local\Temp\ipykernel_26208\189284292.py:5: SyntaxWarning: invalid escape sequence '\S'
  'Server=DAVIDG\SQLEXPRESS01;'


In [4]:
class Order(Hotel):
    global_order_id = 0

    def __init__(self):
        super().__init__()

        Order.global_order_id += 1
        self.order_id = Order.global_order_id   

        self.captain()

    def take_order(self):
        while True:
            ItemName = input("Enter item: ")
            Portion = input("Enter the item type (Half/Full): ").capitalize()
            QTY = int(input("Enter the Quantity: "))

            # Fetch price from MENU
            self.cursor.execute(
                "SELECT Half_price, Full_price FROM dbo.MENU WHERE ItemName = ?",
                (ItemName,)
            )
            price_data = self.cursor.fetchone()

            if not price_data:
                print(" Item not found in menu!")
                continue  

            
            if Portion == "Half":
                price = price_data[0]
            elif Portion == "Full":
                price = price_data[1]
            else:
                print(" Invalid portion! Choose Half or Full.")
                continue

            total = price * QTY

            # Insert in order table
            self.cursor.execute("""
                INSERT INTO Orders 
                (OrderID, ItemName, Portion, Quantity, Price, TotalAmount)
                VALUES (?, ?, ?, ?, ?, ?)
            """, (self.order_id, ItemName, Portion, QTY, price, total))

            self.conn.commit()

            print(" Item added successfully to database")

            more = input("Is it your final order? (Y for Yes | N for No): ").upper()
            if more == 'Y':
                break

In [9]:
from datetime import datetime

class Bill(Order):
    def __init__(self, order_id):
        super().__init__()
        self.order_id = order_id

    def generate_bill(self):
        self.cursor.execute("""
            SELECT ItemName, Portion, Quantity, Price, TotalAmount, OrderDate
            FROM Orders
            WHERE OrderID = ?
        """, (self.order_id,))

        records = self.cursor.fetchall()

        if not records:
            print("No records found")
            return

        # Take date from first row
        order_date = records[0][5]

        print("\n" + "="*50)
        print("           HOTEL SPICE BILL")
        print("="*50)
        print(f"Order ID   : {self.order_id}")
        print(f"Order Date : {order_date}")
        print("-"*50)
        print(f"{'Item':15}{'Portion':10}{'Qty':5}{'Price':10}{'Total':10}")
        print("-"*50)

        grand_total = 0

        for row in records:
            item, portion, qty, price, total, _ = row
            price = int(price)
            total = int(total)
            grand_total += total

            print(f"{item:15}{portion:10}{qty:<5}{price:<10}{total:<10}")

        tax = (grand_total * 5) // 100
        final = grand_total + tax

        print("-"*50)
        print(f"Subtotal     : ₹{grand_total}")
        print(f"GST 5%       : ₹{tax}")
        print(f"Grand Total  : ₹{final}")
        print("="*50)

In [11]:
User_1=Bill(4)
# User_1.Show_menu()
# User_1.take_order()
User_1.generate_bill()

Welcome to the Hotel Spice
What would you like to have: 

           HOTEL SPICE BILL
Order ID   : 4
Order Date : 2026-02-16 09:29:04.547000
--------------------------------------------------
Item           Portion   Qty  Price     Total     
--------------------------------------------------
Shahi Paneer   Half      1    200       200       
Butter Roti    Half      2    30        60        
--------------------------------------------------
Subtotal     : ₹260
GST 5%       : ₹13
Grand Total  : ₹273
